# Using ML for paramter estimation

In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sl
from scipy import stats
from sklearn import datasets
from sklearn import linear_model
from scipy.optimize import minimize
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


"""Loading the Data"""

dir       = 'data/'
file_name = 'grid1_zheb51fo.xlsx'
UX1       = pd.read_excel(dir+file_name, sheet_name='UX1_Index')
UX2       = pd.read_excel(dir+file_name, sheet_name='UX2_Index')
UX1       = UX1.set_index('Date')
UX2       = UX2.set_index('Date')
UX1.dropna(subset = ["PX_LAST"], inplace=True)   #Getting rid of NaN values
UX2.dropna(subset = ["PX_LAST"], inplace=True)
UX1.sort_index(inplace=True)
UX2.sort_index(inplace=True)
dataset = np.array(UX1.PX_LAST)

In [2]:
"""Specifying the Input & Output (Labels)"""

n=0     #Looking at n previous days to estimate paramteres
X=np.array([[dataset[j] for j in range(i, i+n+1)] for i in range(len(dataset) - n)])
Y=np.array([dataset[i+n] for i in range(len(dataset) - n)])

In [3]:
"""Splitting Data into Train and Test set"""

m_training= 2000
m_test= 1000

X_training=X[ :m_training  ]
Y_training=Y[1:m_training+1]   #The labels are shifted 1 to the right since they are tomorrow's value

X_test=X[m_training:m_test]
Y_test=Y[m_training+1:m_test+1]

#for the case of n=0
X_training= np.ravel(X_training)  
X_test    = np.ravel(X_test)

In [4]:
"""Building the Hypothesis"""

def heston_pde_milstein(V0, k, theta, rho, sigma):
    WT  = np.sqrt( 1 ) * np.random.multivariate_normal(np.array([0, 0]), np.array([[1, rho], [rho, 1]]))[1]
    V1 = np.abs(V0+ k * (theta - V0) * 1 + sigma * np.sqrt(V0) * WT + .25 * sigma**2 * (WT**2 - 1))
    return V1

In [5]:
"""Building the Loss Function"""

#The difference between real label and the predicted one to the power of 2
#l = (heston_pde_milstein(X_training[i], r, k, theta, rho, sigma) - Y_training[i])**2

m=len(X_training)  #Training set size
# k: x[0], theta:x[1], rho:x[2], sigma:x[3]

def Ls(X):
    def heston_inner_func(i):   #calculates the predicted lable for each training sample
        WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]
        V1 =  np.abs(X_training[i] + X[0] * (X[0] - X_training[i]) * 1 + 
                     X[3] * np.sqrt(X_training[i]) * WT + .25 * X[3]**2 * (WT**2 - 1))
        return V1
    Ls = (1/m) * np.sum(np.array([(heston_inner_func(i) - Y_training[i])**2 for i in range(m)]))
    return Ls

In [6]:
"""ERM: Empirical Risk Minimization"""

result = minimize(Ls, (1,1,1,1))  #initial values should be given

<ipython-input-5-08901b0a8f86>:11: RuntimeWarning: covariance is not positive-semidefinite.
  WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]
<ipython-input-5-08901b0a8f86>:11: RuntimeWarning: covariance is not positive-semidefinite.
  WT  = np.random.multivariate_normal(np.array([0, 0]), np.array([[1, X[2]], [X[2], 1]]))[1]


In [7]:
best_params = result.x
print("Best Paramteres:", best_params)
print("Minimum Ls:", result.fun)

Best Paramteres: [1. 1. 1. 1.]
Minimum Ls: 258.171208309428


In [ ]:
#Are we trapped inside a loval minima?!!

In [8]:
"""Evaluating the model on test Set"""

k, theta, rho, sigma = best_params[0], best_params[1], best_params[2], best_params[3]
Y_pred = np.array([heston_pde_milstein(X_test[i], k, theta, rho, sigma) for i in range(len(X_test))])
Y_pred = np.ravel(Y_pred)

#True Error
Ld =  (1/len(Y_test)) * np.sum((Y_pred - Y_test)**2)
print(Ld)

353.9178899100621


In [9]:
Y_pred_train = np.array([heston_pde_milstein(X_training[i], k, theta, rho, sigma) for i in range(len(X_training))])
L =  (1/m) * np.sum((Y_pred_train - Y_training)**2)
print(L)

print(Y_pred_train[:50])
print(Y_training[:50])

258.4719600549518
[ 0.3353749   2.48652982 10.71455376  4.08629345  4.25810502  6.47387993
  9.0364508   9.8758356   4.16068681  5.57656333  0.63290172  5.34756914
  6.69915723  9.05107268  0.05254143  2.19888437  1.27835476  2.93862302
  1.32855685  2.36596169  1.41726563  5.36076196  4.22512651  5.97446401
  1.29303034  0.24981222  5.0956642   0.71031502  6.9200246   9.89077004
  0.10582841  4.29703251  4.32232016  2.2368761   4.03058338  0.67795241
 10.8854137   4.411441    3.34273307  0.7300168   4.02636839  0.78199945
  5.16346494  4.80760133  7.29505977  1.4444556   3.80336887  6.95496129
  2.62417655  2.31172405]
[22.25 22.8  22.5  22.   22.55 21.55 21.4  22.65 24.75 24.35 24.55 23.75
 24.   24.65 24.25 23.45 23.55 25.65 25.85 26.2  25.65 25.55 24.95 24.9
 23.6  22.75 21.8  21.35 20.95 21.65 21.1  20.95 20.4  23.05 22.8  22.6
 22.5  21.6  21.25 21.25 21.4  21.6  21.45 21.5  20.9  20.1  19.8  20.2
 19.9  19.45]


In [34]:
"""Using Neural Network for the Whole task: A Non-Physical Experiment"""

#Let's, just for a momemnt, Ignore any pre_knowledge about the subject and see what happens

m_training= 2500
m_test= 500

X_training=X[ :m_training  ]
Y_training=Y[1:m_training+1]   #The labels are shifted 1 to the right since they should be tomorrow's volatility

X_test=X[m_training: m_training + m_test]
Y_test=Y[m_training+1: m_training+m_test+1]


NN_R = MLPRegressor(hidden_layer_sizes=(100,100,100), max_iter=1000, 
                    alpha=1e-4, solver='adam', momentum=0.9,
                    activation='relu', tol=1e-4, learning_rate_init=0.0001)

NN_R.fit(X_training, Y_training)
Y_pred = NN_R.predict(X_test)

# print("Predicted Volatilty:\n", Y_pred[100:120])
# print("\nTrue Volatility:\n",     Y_test[100:120])
# print("\nThe score:", NN_R.score(X_test, Y_test))

NN_R = MLPRegressor(max_iter=1000, alpha=1e-4, solver='adam', momentum=0.9,
                    activation='relu', tol=1e-4, learning_rate_init=0.0001)
parameters = {'hidden_layer_sizes': [(10,), (20,), (40,), (20,20,),(20,20,20), (40,20,10) ]}

gsc = GridSearchCV(NN_R, parameters, cv=6)

gsc.fit(X_training, Y_training)


print("Best parameters set found:")
print(gsc.best_params_)
print("\n")

print("Score with best parameters:")
print(gsc.best_score_)
print("\n")

print("All scores on the grid:")
print(gsc.cv_results_.get('split0_test_score'))
print(gsc.cv_results_.get('split1_test_score'))
print(gsc.cv_results_.get('split2_test_score'))
print(gsc.cv_results_.get('split3_test_score'))
print(gsc.cv_results_.get('split4_test_score'))
print(gsc.cv_results_.get('split5_test_score'))

Best parameters set found:
{'hidden_layer_sizes': (20, 20, 20)}


Score with best parameters:
0.9017655442401683


All scores on the grid:
[0.94466512 0.94130667 0.94460389 0.94348086 0.94227079 0.94479527]
[0.95283685 0.96367    0.96315249 0.96401931 0.96419104 0.96382318]
[0.84072252 0.84765981 0.84523969 0.85733772 0.85515102 0.82448351]
[0.88105096 0.88079457 0.88082584 0.87983886 0.88096221 0.88029017]
[0.85098061 0.85805695 0.85791295 0.84143903 0.85805161 0.85644316]
[0.90951647 0.89321098 0.91094559 0.90860065 0.9099666  0.91064742]
